
# Welcome to StableTuner, Let's get started!
#### This notebook will guide you through the setup process.


__[Join the ST Discord for support, chat and fun times :)](https://discord.gg/DahNECrBUZ)__

#### Start by uploading your payload.zip file (just drag and drop it to the file area) and run this cell as it gets uploaded.

In [ ]:
#Much thanks to IndustriaDitat and entmike for helping making ST linux compatible!
from IPython.display import clear_output
from subprocess import getoutput
installed_xformers = False
GPU_CardName = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')

%pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url "https://download.pytorch.org/whl/cu116"
%pip install -U --pre triton
%pip install ninja bitsandbytes
%pip install -q gradio
if '3090' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/RTX3090-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    installed_xformers = True
if 'A5000' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/A5000-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    installed_xformers = True
if 'T4' in GPU_CardName:
    %pip install https://huggingface.co/r4ziel/xformers_pre_built/resolve/main/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64_t4.whl
    installed_xformers = True
if 'A100' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/A100_13dev/xformers-0.0.13.dev0-py3-none-any.whl
    installed_xformers = True
if 'V100' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/V100_13dev/xformers-0.0.13.dev0-py3-none-any.whl
    installed_xformers = True
if installed_xformers == False:
    clear_output()
    print("No precompiled xformers found for your GPU. Please wait while we compile xformers for your GPU, this might take 20-40 minutes.")
    %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers
%pip install git+https://github.com/huggingface/diffusers.git@0ca1724#egg=diffusers --force-reinstall
clear_output()
print("Done!")

#### Upload finished?, time to run this next cell!

In [ ]:
!unzip -o -q payload.zip
%pip install -r requirements.txt
clear_output()
print("Done!")

#### Looks like you're done installing, let's get training!



In [ ]:
changeMe

## Model Playground

#### This is where you can test your model and package it up.

Run this next cell and select your output model, we'll automatically load the latest checkpoint for you.

In [ ]:
from ipywidgets import widgets
import os
import glob
from IPython.display import clear_output
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from IPython.display import display
import random
import gradio as gr


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print('Loading new model')
        model_path = change['new'] 
        pipe = StableDiffusionPipeline.from_pretrained(model_path,safety_checker=None, torch_dtype=torch.float16).to("cuda")
        scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        pipe.scheduler = scheduler
        print("Checkpoint changed to %s" % change['new'])
        

if 'output' not in os.listdir():
    print("No output folder found. Please run the training cell first.")
models = []
model_dir = os.listdir('output')[0]
output_sort = sorted(glob.iglob('output' + os.sep + model_dir + os.sep+ '*'), key=os.path.getctime, reverse=True)
if len(output_sort) == 0:
    print("No models found in output folder. Please run the training cell first.")
for model in output_sort:
    required_folders = ["vae", "unet", "tokenizer", "text_encoder"]
    if all(x in os.listdir(model) for x in required_folders):
        models.append(model)
model_selection = widgets.Dropdown(
    layout={'width': 'initial'},
    style={'description_width': 'initial'},
    options=models,
    value=models[0],
    # rows=10,
    description='Select Checkpoint:',
    disabled=False
)
model_selection.observe(on_change)
clear_output()
display(model_selection)
model_path = model_selection.value
pipe = StableDiffusionPipeline.from_pretrained(model_path,safety_checker=None, torch_dtype=torch.float16).to("cuda")
scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.scheduler = scheduler
print('Loaded last checkpoint')

All ready?, let's play around with it ;), this next cell will load a small UI for you to generate images and zip it up for download, re-run this cell if you selected a new model!

In [ ]:
def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50,seed=-1,guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
      if seed != -1:
        g_cuda = torch.Generator(device='cuda')
        g_cuda.manual_seed(int(seed))
      else:
        seed = random.randint(0, 100000)
        g_cuda = torch.Generator(device='cuda')
        g_cuda.manual_seed(seed)
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images, seed
def zip_model():
  import shutil
  print('Zipping Model!, Please wait until you see a done message, this can take a few minutes, you can keep generating while you wait!')
  curLocation = os.getcwd()
  model_name = os.path.dirname(model_path)
  shutil.make_archive(model_name,'zip',model_path)
  os.chdir(curLocation)
  print('Done!')
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            with gr.Row():
              run = gr.Button(value="Generate")
              zip = gr.Button(value="Zip Model For Download")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            with gr.Row():
              num_inference_steps = gr.Slider(label="Steps", value=25)
              seed = gr.Number(label="Seed", value=-1)
        with gr.Column():
            gallery = gr.Gallery()
            seedDisplay = gr.Number(label="Used Seed:", value=0)

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps,seed, guidance_scale], outputs=[gallery,seedDisplay])
    zip.click(zip_model)
demo.launch(debug=True)

### This is the end, for now :) ,you can convert your model to CKPT back in StableTuner!.